In [ ]:
using Plots

function ex(performances, distribution; setup::NamedTuple)
    # Extract the performance and arguments from the data structure
    performance = performances[2]
    distargs = performances[1]
    timing_mean = performances[3]

    # Define the segments of the performance data
    segments = [
        (1:81, :blue, "plateau length 20-100"),
        (82:181, :red, "plateau length 100-200"), 
        (182:281, :green, "plateau length 200-300"), 
        (282, :yellow, "plateau length 300-400")
        # hier fehlen noch 400-500 und 500-600(nicht 300-600 !)

    ]

    println(length(performance[1:end]))
    #println("performance ",performance[82:181])

    # Initialize the plot
    plt = plot()

    # Calculate AUC and plot for each segment
    for (range, color, label) in segments
        # Extract segment of performance data
        if isa(range, Int64)
            segment_performance = performance[range:end]
        else 
            segment_performance = performance[range]
        end 

        # Compute ROC points
        FP_rate, TP_rate = roc(segment_performance)

        # Compute AUC for the segment
        AUC_score = auc(FP_rate, TP_rate)

        # Add ROC curve for this segment
        plot!(plt, FP_rate, TP_rate, label="$label AUC score $AUC_score", color=color, lw=3)
    end

    # OVERALL AUC SCORE 
    FP_rate , TP_rate = roc(performance)
    AUC_score = auc(FP_rate, TP_rate)
    plot_title = "ROC measures for $(setup.trial_per_experiment) sequence trials \n of the length $(setup.sequence_length). Trials were balanced: $(setup.balanced)\n from $distribution with: $distargs + $timing_mean \n AUC: $AUC_score"
    plot(plt; title=plot_title, label="ROC curve for plateau length", lw=3, titlefontsize=6)
    plot!(ylims=(0, 100))
    plot!(xlims=(0, 100))
    plot!(xlabelfontsize=7)
    plot!(ylabelfontsize=7)
    plot!(legend=:outerbottom, legendfontsize=7)
    ylabel!("True Positive Rate %")
    xlabel!("False Positive Rate % \n")

end

# experiment setup that is plotted 
experiment_setup = (alphabet = [:A, :B, :C, :D, :d],
         target = target_E1,
         sequence_length=10,
         trial_per_experiment = 600,
         min_tau = 20.0,
         max_tau=600.0,
         step=1.0,
         balanced=true
         )
distname = gamma

plot_ = ex(performances[50], "Normal", setup=experiment_setup)

savefig(plot_ ,"ROC measure COLORFUL WITHOUT JITTER GAUSSIAN BALANCED TARGET_E1.png")
